In [1]:
import requests

url = 'https://apewisdom.io/api/v1.0/filter/all-stocks'

try:
    response = requests.get(url)
    response.raise_for_status()

    data = response.json()
    print("Response JSON:")
    print(data)

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

Response JSON:
{'count': 952, 'pages': 10, 'current_page': 1, 'results': [{'rank': 1, 'ticker': 'SPY', 'name': 'SPDR S&amp;P 500 ETF Trust', 'mentions': 267, 'upvotes': 1409, 'rank_24h_ago': 2, 'mentions_24h_ago': 186}, {'rank': 2, 'ticker': 'TSLA', 'name': 'Tesla', 'mentions': 201, 'upvotes': 848, 'rank_24h_ago': 1, 'mentions_24h_ago': 229}, {'rank': 3, 'ticker': 'NVDA', 'name': 'NVIDIA', 'mentions': 77, 'upvotes': 534, 'rank_24h_ago': 3, 'mentions_24h_ago': 61}, {'rank': 4, 'ticker': 'WOLF', 'name': 'Wolfspeed', 'mentions': 67, 'upvotes': 667, 'rank_24h_ago': 13, 'mentions_24h_ago': 21}, {'rank': 5, 'ticker': 'EU', 'name': 'enCore Energy', 'mentions': 52, 'upvotes': 482, 'rank_24h_ago': 4, 'mentions_24h_ago': 38}, {'rank': 6, 'ticker': 'HOOD', 'name': 'Robinhood', 'mentions': 45, 'upvotes': 317, 'rank_24h_ago': 8, 'mentions_24h_ago': 15}, {'rank': 7, 'ticker': 'QQQ', 'name': 'Invesco QQQ ETF', 'mentions': 43, 'upvotes': 70, 'rank_24h_ago': 5, 'mentions_24h_ago': 42}, {'rank': 8, 'tic

In [2]:
import pandas as pd
df = pd.DataFrame(data['results'])
df.head()

,rank,ticker,name,mentions,upvotes,rank_24h_ago,mentions_24h_ago
0,1,SPY,SPDR S&amp;P 500 ETF Trust,267,1409,2,186.0
1,2,TSLA,Tesla,201,848,1,229.0
2,3,NVDA,NVIDIA,77,534,3,61.0
3,4,WOLF,Wolfspeed,67,667,13,21.0
4,5,EU,enCore Energy,52,482,4,38.0


In [3]:
df[['ticker']]

,ticker
0,SPY
1,TSLA
2,NVDA
3,WOLF
4,EU
...,...
95,SF
96,MD
97,AAA
98,LOVE


In [4]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import re
def get_stock_details(ticker_symbol):

    url = f'https://apewisdom.io/stocks/{ticker_symbol}/'
    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')
    details_div = soup.find('div', class_='row', style='font-size: 18px; margin-top: 20px; margin-bottom: 10px;')

    if details_div:
        details_text = details_div.get_text(separator="\n").strip()
    else:
        details_text = f"Details div not found for {ticker_symbol}."

    match = re.search(r'keywords: (.*)', details_text)
    if match:
        keywords = match.group(1).split(",")
        keywords = [keyword.strip().replace('"', '') for keyword in keywords]
    else:
        keywords = []
    stock_data = yf.Ticker(ticker_symbol)
    try:
        hist = stock_data.history(period="1d")
        if not hist.empty:
            current_price = hist['Close'].iloc[-1]
        else:
            current_price = 'Price not found'
    except Exception as e:
        current_price = f'Error fetching price: {e}'

    positive_percentage = None
    match = re.search(r'(\d+)% positive comments', details_text)
    if match:
        positive_percentage = int(match.group(1))

    return details_text, current_price, positive_percentage, keywords

In [5]:
get_stock_details('AAPL')

('During the last 24H AAPL (Apple) was mentioned 6 times on WallstreetBets by a total of 7 different users.\nApple received 60% positive comments versus 40% negative comments.\nMost comonly the AAPL ticker was associated with the keywords: "planning","move","tsla","goog","earnings"',
 np.float64(209.27999877929688),
 60,
 ['planning', 'move', 'tsla', 'goog', 'earnings'])

In [6]:
get_stock_details('MSFT')

('During the last 24H MSFT (Microsoft) was mentioned 8 times on WallstreetBets by a total of 9 different users.\nMicrosoft received 44% positive comments versus 56% negative comments.\nMost comonly the MSFT ticker was associated with the keywords: "puts","nvda","earnings","slowing","down"',
 np.float64(391.8500061035156),
 44,
 ['puts', 'nvda', 'earnings', 'slowing', 'down'])

In [7]:
get_stock_details('TSLA')

('During the last 24H TSLA (Tesla) was mentioned 161 times on WallstreetBets by a total of 89 different users.\nTesla received 61% positive comments versus 39% negative comments.\nMost comonly the TSLA ticker was associated with the keywords: "bet","before","calls","spy","buy"',
 np.float64(284.95001220703125),
 61,
 ['bet', 'before', 'calls', 'spy', 'buy'])

In [8]:
get_stock_details('GOOGL')

('During the last 24H GOOGL (Alphabet (Google)) was mentioned 9 times on WallstreetBets by a total of 7 different users.\nAlphabet (Google) received 80% positive comments versus 20% negative comments.\nMost comonly the GOOGL ticker was associated with the keywords: "bit","like","goes","monday","dismayed"',
 np.float64(161.9600067138672),
 80,
 ['bit', 'like', 'goes', 'monday', 'dismayed'])

In [9]:
get_stock_details('SPY')

('During the last 24H SPY (SPDR S&P 500 ETF Trust\r\n\r\n) was mentioned 197 times on WallstreetBets by a total of 129 different users.\nSPDR S&P 500 ETF Trust\r\n\r\n received 56% positive comments versus 44% negative comments.\nMost comonly the SPY ticker was associated with the keywords: "bet","before","goes","created","banbet"',
 np.float64(550.6400146484375),
 56,
 ['bet', 'before', 'goes', 'created', 'banbet'])

In [10]:
# for ticker in df['ticker']:
#     print(ticker)
#     print('-----------------------------')
#     # print(get_stock_details(ticker))
#     details, price, positive_pct = get_stock_details(ticker)
#     print(f"\nCurrent stock price: {price}")
#     print(f"Positive Sentiment: {positive_pct}%")
#     print(f"\nSentiment details:\n{details}")
#     print('-----------------------------')

In [13]:
def fetch_details(ticker):
    details, price, positive_pct, keywords = get_stock_details(ticker)
    return pd.Series({
        'current_price': price,
        'positive_sentiment_pct': positive_pct,
        'sentiment_details': details,
        'keywords': keywords
    })

In [14]:
df[['current_price', 'positive_sentiment_pct', 'sentiment_details', 'keywords']] = df['ticker'].apply(fetch_details)

ERROR:yfinance:$DM: possibly delisted; no price data found  (period=1d)
ERROR:yfinance:$BIG: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$MF: possibly delisted; no price data found  (period=1d)
ERROR:yfinance:$OG: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


In [15]:
df[df['mentions']>=10]

,rank,ticker,name,mentions,upvotes,rank_24h_ago,mentions_24h_ago,current_price,positive_sentiment_pct,sentiment_details,keywords
0,1,SPY,SPDR S&amp;P 500 ETF Trust,267,1409,2,186.0,550.640015,56.0,During the last 24H SPY (SPDR S&P 500 ETF Trus...,"[bet, before, goes, created, banbet]"
1,2,TSLA,Tesla,201,848,1,229.0,284.950012,61.0,During the last 24H TSLA (Tesla) was mentioned...,"[bet, before, calls, spy, buy]"
2,3,NVDA,NVIDIA,77,534,3,61.0,111.010002,60.0,During the last 24H NVDA (NVIDIA) was mentione...,"[like, bet, before, img, emote]"
3,4,WOLF,Wolfspeed,67,667,13,21.0,3.27,68.0,During the last 24H WOLF (Wolfspeed) was menti...,"[serious, large, play, happening, re]"
4,5,EU,enCore Energy,52,482,4,38.0,1.6,36.0,During the last 24H EU (enCore Energy) was men...,"[china, deals, expect, retaliation, others]"
5,6,HOOD,Robinhood,45,317,8,15.0,49.450001,72.0,During the last 24H HOOD (Robinhood) was menti...,"[earnings, calls, before, bullish, img]"
6,7,QQQ,Invesco QQQ ETF,43,70,5,42.0,472.559998,50.0,During the last 24H QQQ (Invesco QQQ ETF) was ...,"[dal, dis, twlo, cat, spy]"
7,8,AM,Antero Midstream,37,148,9,29.0,16.879999,75.0,During the last 24H AM (Antero Midstream\r\n) ...,"[before, goes, edt, record, wins]"
8,9,MAGA,ETF Series Solutions Trust - Point Bridge GOP ...,36,712,15,23.0,46.553001,80.0,During the last 24H MAGA (ETF Series Solutions...,"[ashes, fck, yeah, turning, putin]"
9,10,DTE,DTE Energy,32,1018,7,31.0,134.759995,63.0,During the last 24H DTE (DTE Energy\r\n) was m...,"[calls, far, puts, spy, otm]"
